In [ ]:
import json
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import folium

In [ ]:
data = "data/oslo_bydeler_befolkning_2024.geojson"
gdf_bydeler = gpd.read_file(data)
bike_map = folium.Map([59.91273, 10.74609], zoom_start=11)
folium.GeoJson(gdf_bydeler).add_to(bike_map)
bike_map

In [ ]:
from bysykkel_api import get_all_stations
from credentials import credentials

_headers = {
    "Client-Identifier": credentials["user_id"]
}


##response = get_all_stations(_headers)
#stations = response['data']['stations']
#oslo_coords = [59.9139, 10.7522]

def add_station_markers(map, stations):
    for station in stations:
        folium.Marker(
            location=[station['lat'], station['lon']],
            popup=f"<b>{station['name']}</b><br>Capacity: {station['capacity']}",
            icon=folium.Icon(color="blue", icon="bike", prefix="fa")
        ).add_to(map)

In [ ]:
#bike_map

In [ ]:
#[feature["properties"]["bydel"] for feature in geojson_data["features"]]

## Preprocessing

##### Creating Geodataframes

In [ ]:
# Reading the geojson file into a GeoDataFrame
arealer_gdf = gpd.read_file(data)
arealer_gdf.head()

,fid,bydelsnr,bynavn,bydel,befolkning_2024,geometry
0,1,030101,Oslo,Gamle Oslo,63721,"MULTIPOLYGON (((10.66087 59.88365, 10.66472 59..."
1,2,030102,Oslo,Grünerløkka,65532,"MULTIPOLYGON (((10.75378 59.91469, 10.75453 59..."
2,3,030103,Oslo,Sagene,47627,"MULTIPOLYGON (((10.77156 59.93095, 10.77135 59..."
3,4,030104,Oslo,St.Hanshaugen,41571,"MULTIPOLYGON (((10.75136 59.91923, 10.75158 59..."
4,5,030105,Oslo,Frogner,60727,"MULTIPOLYGON (((10.65562 59.89038, 10.65381 59..."


In [ ]:
# Calling the station method and creating a GeoDataFrame
from bysykkel_api import get_all_stations
from credentials import credentials

_headers = {
    "Client-Identifier": credentials["user_id"]
}

response = get_all_stations(_headers)
stations = response['data']['stations']

In [ ]:
stations_gdf = gpd.GeoDataFrame(stations)
print(stations_gdf.shape)
stations_gdf.head()

(255, 10)


,station_id,name,address,cross_street,lat,lon,is_virtual_station,capacity,station_area,rental_uris
0,5220,Blindern vgs,Sognsveien,Under broen,59.949881,10.730158,False,20,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5220', 'i..."
1,5217,Egertorget,Øvre Slottsgate 19,ved Max burger,59.912674,10.742025,False,30,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5217', 'i..."
2,5113,Leirfallsgata,Markveien 58,ved Markveien,59.919115,10.757831,False,15,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5113', 'i..."
3,4683,Valle Vision,Valle Vision,Valle,59.916065,10.807178,False,21,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4683', 'i..."
4,4552,Bryn Brannstasjon,Agmund Bolts vei 4,Ved brannstasjonen,59.912716,10.813758,False,18,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4552', 'i..."


In [ ]:
from bysykkel_api import get_bike_info
availabiliy_info = get_bike_info(_headers)
availability_df = pd.DataFrame(availabiliy_info['data']['stations'])


In [ ]:
print(availability_df.shape)
availability_df.head()

(255, 9)


,station_id,is_installed,is_renting,is_returning,last_reported,num_vehicles_available,num_bikes_available,num_docks_available,vehicle_types_available
0,5220,True,True,True,1733067839,0,0,18,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."
1,5217,True,True,True,1733067839,27,27,0,"[{'vehicle_type_id': 'bike', 'count': 27}, {'v..."
2,5113,True,True,True,1733067839,1,1,8,"[{'vehicle_type_id': 'bike', 'count': 1}, {'ve..."
3,4683,True,True,True,1733067839,0,0,20,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."
4,4552,True,True,True,1733067839,0,0,13,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."


In [ ]:
def match_stations_and_availability(stations_gdf, availability_df):
    stations_gdf = stations_gdf.merge(availability_df, left_on="station_id", right_on="station_id")
    return stations_gdf

stations_df = match_stations_and_availability(stations_gdf, availability_df)
# Turn stations_df into gdf, where lat and lon are used to create a geometry column
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.lon, stations_df.lat))



MergeError: Passing 'suffixes' which cause duplicate columns {'num_docks_available_x', 'num_bikes_available_x', 'num_vehicles_available_x', 'vehicle_types_available_x', 'is_installed_x', 'is_returning_x', 'last_reported_x', 'is_renting_x'} is not allowed.

Now each row contains all the necessary information for the map, and analysis plus visualization can be done. The next step is to create the map witht the polygons for Oslo and the points for the stations. 

In [ ]:
gdf_bydeler.shape

(17, 6)

In [ ]:
points_in_polygons = gpd.sjoin(stations_gdf, gdf_bydeler, how="inner", predicate="within")
points_in_polygons.head()

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>